In [ ]:
# Importing stock ml libraries  (#Requirements)
!pip install seqeval
!pip install transformers
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from torch import cuda
from seqeval.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report
from google.colab import drive

def requirements():
  print("numpy ",np.__version__)
  print("pandas ",pd.__version__)
  print("sklearn ",sklearn.__version__)
  print("torch ",torch.__version__)
  print("transformers", transformers.__version__)

In [ ]:
def te_sentiment(rating):
  rating = int(rating)
  if rating == 2:
    return 'I'
  elif rating == 1:
    return 'B'
  else:
    return 'O'

In [ ]:
def te_target(rating):
  rating = str(rating)
  if rating == 'I':
    return 3
  elif rating == 'B':
    return 2
  else:
    return 1

In [ ]:
def process_data(df):

    df_blank1 = pd.DataFrame()
    df_blank1['temp'] = 0
    df_blank1['temp'] = df_blank1['temp'].astype('object')
    df_blank2 = pd.DataFrame()
    df_blank2['target'] = 0
    df_blank2['target'] = df_blank2['target'].astype('object')

    enc_tag = preprocessing.LabelEncoder() 
    sentences = df.groupby("Sentence")["Word"].apply(list).values
    #print(len(sentences))
    counter = 0
    for i in sentences:
      #print(i)
      df_blank1.at[counter, 'temp'] = np.array(i)
      counter = counter + 1
    tag = df.groupby("Sentence")["target"].apply(list).values
    c = 0
    for i in tag:
      #print(i)
      df_blank2.at[c, 'target'] = np.asarray(i)
      c = c + 1

    df_test = pd.concat([df_blank1,df_blank2], axis=1)
    
    return sentences, tag, enc_tag, df_test

In [ ]:
def data_loader(data_test_category, data_test_sentense, data_test_target):
  df_test_category = pd.read_csv(data_test_category)
  if('aspectCategories' in  df_test_category):
    df_test_category = df_test_category.drop(['aspectCategories'], axis=1)
  df_test_category = df_test_category.reset_index(drop=True)
  df_test_category['list'] = df_test_category[df_test_category.columns[1:]].values.tolist()
  df_test_category = df_test_category[['text', 'list']].copy()
  print("-----------------------------------------------------")
  print(df_test_category.head(5)) 

  df_test_sentense = pd.read_csv(data_test_sentense, sep='delimiter', names=["Sentence1"])
  df_test_target = pd.read_csv(data_test_target, sep='delimiter', names=["target1"])
  df_test_combo = pd.concat([df_test_sentense, df_test_target], axis=1)

  print("-----------------------------------------------------")
  print(df_test_combo.head(5))

  u = df_test_combo.Sentence1.str.split(expand=True).stack()
  v = df_test_combo.target1.str.split(expand=True).stack()

  df_test_combo = pd.DataFrame({
        'Sentence': u.index.get_level_values(0) + 1, 
        'Word': u.values, 
        'target': v.values,
        #'Entity': u.map(dict(zip(df.value, df.entity))).fillna('Object').values
    })
  print("-----------------------------------------------------")
  print(df_test_combo.head(5))

  df_test_combo['target'] = df_test_combo.target.apply(te_sentiment)

  print("-----------------------------------------------------")
  print(df_test_combo.head(5))

  df_test_combo['target'] = df_test_combo.target.apply(te_target)

  print("-----------------------------------------------------")
  print(df_test_combo.head(5))

  sentences_te, tag_te, enc_tag_te, df_test = process_data(df_test_combo) 

  df_test = pd.concat([df_test, df_test_category], axis=1)

  
  return df_test



 
  

  

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.temp
        
        self.tag = self.data.target
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        tags = self.tag[index]
        targets = self.targets[index]

        ids = []
        target_tag =[]
        #target =[]
        MAX_LEN = 100
        for i, s in enumerate(text):
            inputs = self.tokenizer.encode(
                s,
                add_special_tokens=False
            )
           
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [101] + ids + [102]
        target_tag = [1] + target_tag + [1]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        #marked_text = "[CLS] " + text + " [SEP]"
        #tokenized_text = self.tokenizer.encode(text)
        '''
        print(text)
        print("***********************")
        #print(tokenized_text)
        #print("***********************")
        print(tags)
        print("***********************")
        print(ids)
        print("***********************")
        print(mask)
        print("***********************")
        print(token_type_ids)
        print("***********************")
        print(target_tag)
        print("***********************")
        print(targets)
        print("***********************")
        '''

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
class BERTClass(torch.nn.Module, ):
    def __init__(self, numclasses,numlayers,num_heads, embeddim, hiddendim_lstm, fchidden):
        super(BERTClass, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Dropout(0.3)
        self.l4 = torch.nn.Linear(768, 4)
        self.l5 = torch.nn.Linear(768, 5)
    
    def forward(self, ids, mask, token_type_ids):
        o1, pooled_output = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        #print(output_2.shape)
        output_2 = self.l2(o1)
        #print(output_2.shape)
        output_3 = self.l3(pooled_output)
        #output_2 = self.lstm(output_2, None)
        #output_2 = self.conv(output_2)
        #output_2 = self.conv(output_2)
        #print(output_2.shape)
        output = self.l4(output_2)
        
        out = self.l5(output_3)
        return output, out



class Bert_LSTM(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_LSTM, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.1)

        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm*2, bidirectional = True, batch_first=False)
        #self.conv = torch.nn.Conv1d( 100, self.hiddendim_lstm, 3, stride=2)
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=False)
        #self.lstm = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False) # noqa
        #self.attention = torch.nn.MultiheadAttention(self.hiddendim_lstm*2, num_heads = 2)
        self.fc = torch.nn.Linear(self.hiddendim_lstm, 4)
        self.fc1 = torch.nn.Linear(self.hiddendim_lstm, 5)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.lstm(last_hidden_state, None)
      #print(out.shape)
      #attn_output, attn_output_weights = self.attention(out, out, out)
      #print(attn_output.shape)
      #print(attn_output_weights.shape)
      NER = self.dropout(out)
      ACC = self.dropout(out[:, -1, :])
      #print(out.shape)
      output = self.fc(NER)
      out1 = self.fc1(ACC)
      
      return output, out1




class Bert_GRU(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_GRU, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.1)

        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm*2, bidirectional = True, batch_first=False)
        #self.conv = torch.nn.Conv1d( 100, self.hiddendim_lstm, 3, stride=2)
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False)
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=False) # noqa
        #self.attention = torch.nn.MultiheadAttention(self.hiddendim_lstm*2, num_heads = 2)
        self.fc = torch.nn.Linear(self.hiddendim_lstm, 4)
        self.fc1 = torch.nn.Linear(self.hiddendim_lstm, 5)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.Gru(last_hidden_state, None)
      #print(out.shape)
      #attn_output, attn_output_weights = self.attention(out, out, out)
      #print(attn_output.shape)
      #print(attn_output_weights.shape)
      NER = self.dropout(out)
      ACC = self.dropout(out[:, -1, :])
      #print(out.shape)
      output = self.fc(NER)
      out1 = self.fc1(ACC)
      
      return output, out1



class Bert_Attention(torch.nn.Module):
    def __init__(self, numclasses, device):
        super(Bert_Attention, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.fchidden = fchidden
        self.dropout = torch.nn.Dropout(0.1)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hiddendim_lstm*2))
        self.q = torch.nn.Parameter(torch.from_numpy(q_t)).float().to(device)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hiddendim_lstm*2, self.fchidden)) # noqa
        self.w_h = torch.nn.Parameter(torch.from_numpy(w_ht)).float().to(device)

        self.fc = torch.nn.Linear(self.fchidden, 4)
        self.fc1 = torch.nn.Linear(self.fchidden, 5)


    def forward(self, inp_ids, att_mask, token_ids):
        last_hidden_state, pooler_output = self.bert(input_ids=inp_ids, attention_mask=att_mask, token_type_ids=token_ids)
        #print("1",last_hidden_state.shape)
        gru, _ = self.Gru(last_hidden_state, None)
        #print("2",out.shape)
        #out = self.attention(out)
        att = self.attention(gru)
        #print("3",out.shape)
        out = self.dropout(att)
        print("la",out.shape)
        output = self.fc(out)
        out1 = self.fc1(out)
        #print("la",out.shape)
        return output, out1

    def attention(self, h):
        #print("hhhhh",h.shape)
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = torch.nn.functional.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        #print("tem",v_temp.shape)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        #print(v.shape)
        return v


In [ ]:
def loss_fn(output, target, mask, num_labels):
    lfn = torch.nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)[active_loss]
    #active_labels = torch.where(
    #    active_loss,
    #    target.view(-1),
    #    torch.tensor(lfn.ignore_index).type_as(target)
    #)
    active_labels = target.view(-1)[active_loss]
    #print(active_logits.shape)
    #print(active_labels.shape)
    loss = lfn(active_logits, active_labels)
    return loss
    
def loss_fun(outputs, targets):
    #print(outputs.shape)
    #print(targets.shape)
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def evaluate(loader, model, device):
    model.eval()
    with torch.no_grad():
        loss = 0.0
        y_pred = []
        y_true = []
        for _,data in enumerate(loader, 0):              
            #print(data)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets_tag = data['target_tag'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs, out = model(ids, mask, token_type_ids)
            loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
            loss2 = loss_fun(out, targets)
            loss = loss1 + loss2
            preds = torch.sigmoid(out)
            y_pred.extend(preds.tolist())
            y_true.extend(targets.tolist())
            y_predict = np.array(y_pred) >= 0.5

        F1 = round((metrics.accuracy_score(y_true, y_predict)), 2) * 100
        return loss, F1

In [ ]:
def train(epoch, model, training_loader, device,optimizer, num_tag):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets_tag = data['target_tag'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs, out = model(ids, mask, token_type_ids)
        #print(outputs.shape)
        #print(out.shape)
        optimizer.zero_grad()
        loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
        loss2 = loss_fun(out, targets)
        loss = loss1 + loss2
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}, Loss1:  {loss1.item()}, Loss2:  {loss2.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validation(epoch, model, device, testing_loader,num_tag):
    model.eval()
    fin_targets=[]
    fin_targets_tag=[]
    fin_outputs=[]
    fin_out=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets_tag = data['target_tag'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs, out = model(ids, mask, token_type_ids)
            loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
            loss2 = loss_fun(out, targets)
            loss = loss1 + loss2
            if _%5000==0:
                print(f'Epoch: {epoch}, Loss:  {loss.item()}, Loss1:  {loss1.item()}, Loss2:  {loss2.item()}')
            #outputs = torch.log(outputs)
            #outputs = torch.cat(outputs, dim=1).flatten()
            #print(outputs)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_targets_tag.extend(targets_tag.cpu().detach().numpy().tolist())
            fin_out.extend(torch.sigmoid(out).cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets, fin_targets_tag, fin_out

In [ ]:
def validate(EPOCHS,model, device, testing_loader,num_tag):
  best_accuracy = 0
  predictions = []
  truetarget = []
  for epoch in range(EPOCHS):
      outputs, targets, targets_tag, out = validation(epoch, model, device, testing_loader,num_tag)
      print(out[1])
      print(targets[1])
      for p, r in zip(outputs, targets_tag):
        #print(p)
        #print(r)
        outs = []
        tar = []
        for m, n in zip (p,r):
          if (n != 0):
            a = m.index(max(m))
            #print(a)
            tar.append(n)
            outs.append(a)
            #print(outs)
        predictions.append(outs)
        truetarget.append(tar)
          
      print(predictions[1])
      print(truetarget[1])
      print(outputs[1])
      print(targets_tag[1]) 
      out = np.array(out) >= 0.5
      #accuracy = metrics.accuracy_score(targets_tag, outputs)
      accuracy = metrics.accuracy_score(targets, out)
      f1_score_micro = metrics.f1_score(targets, out, average='micro')
      f1_score_macro = metrics.f1_score(targets, out, average='macro')
      #precision_score_micro = metrics.precision_score(targets, outputs, average='micro')
      precision_score_macro = metrics.precision_score(targets, out, average='macro')
      #recall_score_micro = metrics.recall_score(targets, outputs, average='micro')
      recall_score_macro = metrics.recall_score(targets, out, average='macro')
      print(f"Accuracy Score = {accuracy}")
      print(f"F1 Score (Micro) = {f1_score_micro}")
      print(f"F1 Score (Macro) = {f1_score_macro}")
      #print(f"precision_score (Micro) = {precision_score_micro}")
      print(f"precision_score (Macro) = {precision_score_macro}")
      #print(f"recall_score (Micro) = {recall_score_micro}")
      print(f"recall_score (Macro) = {recall_score_macro}")
      if accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = accuracy

      return predictions, truetarget, out, targets

In [ ]:
def report(predictions, truetarget):
  #print("F1-Score: {}".format(f1_score(pred, targ)))
  print("Validation Accuracy: {}".format(accuracy_score(predictions, truetarget)))
  pred = []
  targ = []
  for p in predictions:
    #print(p)
    q = []
    for p_i in p:
      #print(p_i)
      if p_i == 1:
        q.append('O')
        #print(p_i)
      elif p_i == 2:
        q.append('B')
        #print(p_i)
      else:
        q.append('I')
    pred.append(q)
      #print(predictions)
      #print(pred)
  for p in truetarget:
    #print(p)
    q = []
    for p_i in p:
      #print(p_i)
      if p_i == 1:
        q.append('O')
        #print(p_i)
      elif p_i == 2:
        q.append('B')
        #print(p_i)
      else:
        q.append('I')
    targ.append(q)
  print(pred)
  print(targ)
  class_names = ['Negative', 'Neutral', 'Positive']
  print("F1-Score: {}".format(classification_report(pred, targ)))
  print("F1-Score: {}".format(f1_score(pred, targ)))


  print("Validation Accuracy: {}".format(accuracy_score(pred, targ)))

In [79]:
def main():
    
    print("Requirements: ")
    requirements()
    print("-----------------------------------------------------")


    device = 'cuda' if cuda.is_available() else 'cpu'
    print("Device = ", device)
    drive.mount('/content/drive/')
    datapath= '/content/drive/MyDrive/Bert_mtl/Data/Data_For_ATE_and_ACD/'

    data_test_category = datapath + 'Test/MTLBACategory.csv'
    data_test_sentense = datapath + 'Test/sentence (1).txt'
    data_test_target = datapath + 'Test/target (1).txt'

    data_train_category = datapath + 'Train/semeval2014.csv'
    data_train_sentense = datapath + 'Train/sentence.txt'
    data_train_target = datapath + 'Train/target.txt'

    df_test = data_loader(data_test_category, data_test_sentense, data_test_target)
    df_train = data_loader(data_train_category, data_train_sentense, data_train_target)

    print("-----------------------------------------------------")
    print(df_test.head(5))


    print("-----------------------------------------------------")
    print(df_train.head(5))

    # Sections of config

    # Defining some key variables that will be used later on in the training
    MAX_LEN = 100
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 4
    EPOCHS = 4
    LEARNING_RATE = 2e-05
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_dict=False)


    train_size = 1.00
    test_size = 1.0
    train_dataset=df_train.sample(frac=train_size,random_state=200)
    test_dataset=df_test.sample(frac=test_size,random_state=200)
    train_dataset = train_dataset.reset_index(drop=True)

    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(test_dataset.shape))

    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
    testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)


    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    training_loader = DataLoader(training_set, **train_params)
    testing_loader = DataLoader(testing_set, **test_params)


    fchidden = 256
    hiddendim_lstm = 256
    embeddim = 768
    numlayers = 12
    numclasses = 4
    num_heads = 2

    model = BERTClass(numclasses,numlayers,num_heads, embeddim , hiddendim_lstm, fchidden)
    model.to(device)
    print("-----------------------------------------------------")
    model

    print("-----------------------------------------------------")

    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    import joblib

    #meta_data = {
    #    "enc_tag": enc_tag
    #}

    #joblib.dump(meta_data, "meta.bin")

    #num_tag = len(list(enc_tag.classes_))
    num_tag = 4
    

    for epoch in range(EPOCHS):
      train(epoch, model, training_loader,device,optimizer,num_tag)

    print("-----------------------------------------------------")
    predictions, truetarget, out, targets = validate(EPOCHS, model,device, testing_loader,num_tag)


    target_names = ['service', 'food', 'anecdotes/miscellaneous', 'price', 'ambiance']
    print("-----------------------------------------------------")
    print(classification_report(out, targets, target_names))

    print("-----------------------------------------------------")
    #report(predictions, truetarget)


if __name__ == '__main__':
    main()

Requirements: 
numpy  1.21.6
pandas  1.3.5
sklearn  1.0.2
torch  1.10.0+cu111
transformers 4.18.0
-----------------------------------------------------
Device =  cuda
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
-----------------------------------------------------
                                                text             list
0                    The bread is top notch as well.  [0, 1, 0, 0, 0]
1  I have to say they have one of the fastest del...  [1, 0, 0, 0, 0]
2        Food is always fresh and hot- ready to eat!  [0, 1, 0, 0, 0]
3      Did I mention that the coffee is OUTSTANDING?  [0, 1, 0, 0, 0]
4  Certainly not the best sushi in New York, howe...  [0, 1, 0, 0, 1]
-----------------------------------------------------
                                           Sentence1  \
0                     the bread is top notch as well   
1  i have to say they have one of the fastest del...   
2     

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


-----------------------------------------------------
                                                text             list
0                but the staff was so horrible to us  [1, 0, 0, 0, 0]
1  to be completely fair the only redeeming facto...  [0, 1, 1, 0, 0]
2  the food is uniformly exceptional with a very ...  [0, 1, 0, 0, 0]
3  where gabriela personaly greets you and recomm...  [1, 0, 0, 0, 0]
4  for those that go once and dont enjoy it all i...  [0, 0, 1, 0, 0]
-----------------------------------------------------
                                           Sentence1  \
0                but the staff was so horrible to us   
1  to be completely fair , the only redeeming fac...   
2  the food is uniformly exceptional , with a ver...   
3  where gabriela personaly greets you and recomm...   
4  for those that go once and do n't enjoy it , a...   

                                             target1  
0                                    0 0 1 0 0 0 0 0  
1  0 0 0 0 0 0 0 0 0 0 0 

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


-----------------------------------------------------
                                                temp  \
0             [the, bread, is, top, notch, as, well]   
1  [i, have, to, say, they, have, one, of, the, f...   
2  [food, is, always, fresh, and, hot-, ready, to...   
3  [did, i, mention, that, the, coffee, is, outst...   
4  [certainly, not, the, best, sushi, in, new, yo...   

                                              target  \
0                              [1, 2, 1, 1, 1, 1, 1]   
1      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1]   
2                     [2, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
3                           [1, 1, 1, 1, 1, 2, 1, 1]   
4  [1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                                text             list  
0                    The bread is top notch as well.  [0, 1, 0, 0, 0]  
1  I have to say they have one of the fastest del...  [1, 0, 0, 0, 0]  
2        Food is always fresh and hot- ready to eat!  [0

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-----------------------------------------------------
-----------------------------------------------------
Epoch: 0, Loss:  2.2425453662872314, Loss1:  1.4859684705734253, Loss2:  0.7565769553184509
Epoch: 1, Loss:  0.4124625325202942, Loss1:  0.07674767822027206, Loss2:  0.33571484684944153
Epoch: 2, Loss:  0.12883272767066956, Loss1:  0.03634382411837578, Loss2:  0.09248889982700348
Epoch: 3, Loss:  0.18832442164421082, Loss1:  0.059120092540979385, Loss2:  0.12920433282852173
-----------------------------------------------------
Epoch: 0, Loss:  0.15201054513454437, Loss1:  0.1351136416196823, Loss2:  0.01689690724015236
[0.010009605437517166, 0.977443516254425, 0.05494960770010948, 0.009500931017100811, 0.008510638028383255]
[0.0, 1.0, 0.0, 0.0, 0.0]
[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[0.04280104860663414, 0.9916508793830872, 0.1403278261423111, 0.26926183700561523], [0.

UFuncTypeError: ignored